In [ ]:
###Alper Beykan YOLCU###
###161180072###
###CENG 471 Assignment2###
###December 2020###

In [ ]:
"""
The goal of this assignment is to implement a Canny Edge Detector from scratch using Python.
Important Note: For this assignment, you are not allowed to use the Canny Edge Detector methods
from vision libraries such as OpenCV. You can use other methods and operators from these libraries
for other aspects of the homework such as image uploading, writing to a file, grayscale conversion,
applying Gaussian filter, matrix operations etc. You can use the image provided in the assignment
folder.
Canny edge detector:
The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a
wide range of edges in images. It was developed by John F. Canny in 1986. Among the edge detection
methods developed so far, Canny edge detection algorithm is one of the most strictly defined methods
that provides good and reliable detection, and is therefore among the most popular methods. [1]
"""

In [ ]:
import cv2
import math
import numpy as np
import skimage as sk
import skimage.feature
import skimage.io as skio
import skimage.transform as sktr
from scipy.signal import convolve2d
from skimage.io import imread, imshow
import matplotlib.pyplot as plt

In [ ]:
def showIm(img):#image show function
    
    cv2.imshow("showing image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 0

In [ ]:
def plotIm(img):#image plot function.
    plt.imshow(img,cmap=plt.get_cmap('gray')) 
    return plt.show()

In [ ]:
#1. Grayscale Conversion (5 points)
#due to week8_edgeandboundarydetection lesson..
def grayscale_Conversion(img):#converts img to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) 
    print("Grayscaled img")
    return img  
    

In [ ]:
#2. Apply Gaussian filter to smooth the image in order to remove the noise (20 points)
#You can determine
#the kernel size and sigma value through trial and error.
def apply_GuassianFilter(img):#The blur removes some of the noise before further processing the image.
    img = cv2.GaussianBlur(img, (3, 3), 1) 
    print("GuassianFiltered img")
    return img

In [ ]:
def Normalize(img):##Normalizing the image after operated, not necessery for all operations
    img = img/np.max(img)
    return img

In [ ]:
#3. Find the intensity gradients of the image (30 points)
#The Gradient calculation step detects the edge intensity and direction. The gradients can be determined by using a Sobel filter.
#Then, the magnitude G and the slope θ of the gradient are calculated as follow 
#(the derivativesIx and Iy with respect to x and y are calculated by convolving Image (I) with Sobel kernels in x and y directions):
def conv_transform(image):##performs matrix operation for colvolving function
    image_copy = image.copy()

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image_copy[i][j] = image[image.shape[0]-i-1][image.shape[1]-j-1]
    return image_copy

def convolve(image, kernel):#convolves the x and y kernels over the image.
    kernel = conv_transform(kernel)
    height,width = image.shape[:2]
    kernel_h,kernel_w = kernel.shape[:2]

    h = kernel_h//2#h
    w = kernel_w//2#w

    image_conv = np.zeros(image.shape)

    for i in range(h, height-h):
        for j in range(w, width-w):
            sum = 0

            for m in range(kernel_h):
                for n in range(kernel_w):
                    sum = (sum + kernel[m][n] * image[i-h+m][j-w+n])
            
            image_conv[i][j] = sum

    return image_conv


def sobel(img):
    #first we convolve kernels and image by convolve function and we get x and y gradients.
    #then we use that arrays for calculating magnitude and the slope of gradient
    
    # Prewitt operators for Sobel Kernel to find gradient of Y and X
    x_kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])#for hprizonal direction
    y_kernel = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])#for vertical direction
    gx = convolve(img, x_kernel)
    gy = convolve(img, y_kernel)
    magnitudeG = np.hypot(gx,gy)
    MagnitudeG=Normalize(magnitudeG)
    plotIm(magnitudeG)
    g_angle=np.degrees(np.arctan2(gy,gx))
    return magnitudeG,g_angle


In [ ]:
#Apply non-maximum suppression to get rid of unnecessary edges (10 points)
#We apply non-maximum suppression to remove redundant/duplicate edges identified by
#Sobel Edge. We want just one line to show the edge rather than having multiple lines for the
#same edge. This can be done by the Non-Max Suppression Algorithm. Please see [1] for details.
def NonMaxSup(mg, ag):
    #we are scanning the image's intense matrix. the goal is get rid of non dominant pixels
    #the scanning is happening on different lines. if the pixel is intensed it stays, if not we are assigning 0 to its value
    img = np.zeros(mg.shape)
    for i in range(1, int(mg.shape[0]) - 1):
        for j in range(1, int(mg.shape[1]) - 1):
            if((ag[i,j] >= -22.5 and ag[i,j] <= 22.5) or (ag[i,j] <= -157.5 and ag[i,j] >= 157.5)):
                if((mg[i,j] > mg[i,j+1]) and (mg[i,j] > mg[i,j-1])):
                    img[i,j] = mg[i,j]
                else:
                    img[i,j] = 0
            if((ag[i,j] >= 22.5 and ag[i,j] <= 67.5) or (ag[i,j] <= -112.5 and ag[i,j] >= -157.5)):
                if((mg[i,j] > mg[i+1,j+1]) and (mg[i,j] > mg[i-1,j-1])):
                    img[i,j] = mg[i,j]
                else:
                    img[i,j] = 0
            if((ag[i,j] >= 67.5 and ag[i,j] <= 112.5) or (ag[i,j] <= -67.5 and ag[i,j] >= -112.5)):
                if((mg[i,j] > mg[i+1,j]) and (mg[i,j] > mg[i-1,j])):
                    img[i,j] = mg[i,j]
                else:
                    img[i,j] = 0
            if((ag[i,j] >= 112.5 and ag[i,j] <= 157.5) or (ag[i,j] <= -22.5 and ag[i,j] >= -67.5)):
                if((mg[i,j] > mg[i+1,j-1]) and (mg[i,j] > mg[i-1,j+1])):
                    img[i,j] = mg[i,j]
                else:
                    img[i,j] = 0

    return Normalize(img)

In [ ]:
#5. Apply double threshold to determine potential edges. (5 points)
#You can set two thresholds, a high and a low threshold to filter out edge pixels with a weak
#gradient value and preserve edge pixels with a high gradient value.
def double_TH(img):
    #we are defining a high and low threshhold values.
    #then we locate the high and low threshhold values and replacing them with defined values.
    #the reason we do that similar to previous step, detecting irrelevent pixels.
    highThreshold = img.max() * 0.225;
    lowThreshold = highThreshold * 0.1;
    result = np.zeros(img.shape[:2])#empty img for writing on it
    weak = np.int32(75)
    strong = np.int32(250)
    strong_i, strong_j = np.where(img >= highThreshold)
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))

    result[strong_i, strong_j] = strong
    result[weak_i, weak_j] = weak
    return result

In [ ]:
#6. Track edge by hysteresis: (5 points) Finalize the detection of edges by suppressing all the other
#edges that are weak and not connected to strong edges. (Fort his stage, partial solutions are
#okay, you don’t need to consider all cases). Please see [1] for details.
#For example, you can loop through each pixel in the image, if the value of the pixel is weak and
#there are not any neighboring pixel with value 255, you can set the value of the pixel to 0.
#Report (25 points)
def hysteresis(img):
    #we are scanning through the whole image, if there is a weak pixel we look around that pixel
    #if there is any high pixel around it we are connect it to the high pixelled value by 
    #assigning same value with high pixels. if there is no connection between a weak pixel and 
    #a high pixel we are assigning its value to 0 and getting rid of this pixel.
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            if (img[i,j] == 75):
                if 255 in [img[i+1, j-1],img[i+1, j],img[i+1, j+1],img[i, j-1],img[i, j+1],img[i-1, j-1],img[i-1, j],img[i-1, j+1]]:
                    img[i, j] = 255
                else:
                    img[i, j] = 0
    return img

In [ ]:
def apply_Canny(img):
    ##the algorithm works on 6 steps
    ##first we convert the image to grayscale so we can perform operations on the image easily
    ##second we filter the image with guassian blur to reduce noise of the image.
    #third we calculate the gradients of the image so we can calculate the magnitude and the slope of it
    #fourth we use the magnitude and the slope values which we obtain from sobel operations for Non-Max Suppression.
    #the NMS roams through the image on different aspects and returns an image that contains intense pixels.
    #fifth we apply double threshholding on the image so we can detect the high and low pixels.
    #at last we track the edges based on image whether high or low so we reduce unnecessery pixels for edge detection.
    showIm(img)
    img=grayscale_Conversion(img)
    showIm(img)
    img=apply_GuassianFilter(img)
    showIm(img)
    mG,aG=sobel(img)
    img=NonMaxSup(mG,aG)
    plotIm(img)
    double_TH(img)
    plotIm(img)
    hysteresis(img)
    plotIm(img)
    return 0


In [ ]:
img=cv2.imread("C:/Users/user/Desktop/dersler/imageProcessing/assignment_2/Assignment 2/chihuahua-dog-puppy-cute-39317_small.jpeg")
apply_Canny(img)

In [ ]:
#References:
#1: https://en.wikipedia.org/wiki/Canny_edge_detector
#2: Image Ref : The image used for the assignment is from https://images.pexels.com/photos/39317/chihuahua-dog-puppy-cute39317.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
#3: https://towardsdatascience.com/canny-edge-detection-step-by-step-in-python-computer-vision-b49c3a2d8123
#4: https://github.com/sivomke/Edge-detection/blob/master/main.py